In [5]:
import geopandas as gpd
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from shapely.geometry import Polygon
from shapely.geometry import Point
import cartopy.crs as ccrs
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy
import cartopy.feature as cfeature
import json
import s3fs

%config InlineBackend.figure_format='retina'

In [6]:
catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

In [9]:
catalog.iloc[4]['zarr_url']

'http://its-live-data.s3.amazonaws.com/datacubes/v02/S50W070/ITS_LIVE_vel_EPSG32718_G0120_X550000_Y4250000.zarr'

In [15]:
ds = xr.open_dataset(catalog.iloc[10]['zarr_url'], engine='rasterio', chunks='auto')

RasterioIOError: HTTP response code: 404